In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode,split
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, LongType, DoubleType, FloatType, DataType, DateType, TimestampType

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
spark = (SparkSession
 .builder
 .appName("Task_1")
 .getOrCreate())

22/09/06 11:39:45 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.197.132 instead (on interface ens33)
22/09/06 11:39:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/06 11:39:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/06 11:39:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/06 11:39:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/06 11:39:47 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/09/06 11:39:47 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/09/06 11:39:47

In [3]:
"""
Defining the schema and the datatypes of the columns

"""

sc = StructType([
        StructField('analytics', StructType([
             StructField('clicks', IntegerType(), True),
             StructField('impressions', IntegerType(), True)
             ])),
         StructField('datetime', TimestampType(), True),
         StructField('sales', StructType([
             StructField('quantity', IntegerType(), True),
             StructField('total_price', DoubleType(), True)
             ]))
         ])

print(sc)



StructType(List(StructField(analytics,StructType(List(StructField(clicks,IntegerType,true),StructField(impressions,IntegerType,true))),true),StructField(datetime,TimestampType,true),StructField(sales,StructType(List(StructField(quantity,IntegerType,true),StructField(total_price,DoubleType,true))),true)))


In [4]:
json_file_path = "/home/ubuntu/Documents/Spark_Streaming/python_file/data/json_files"
csv_file_path = "/home/ubuntu/Documents/Spark_Streaming/python_file/data/csv_files/"
csv_checkpoint_path = "/home/ubuntu/Documents/Spark_Streaming/python_file/data/checkpoint"

In [5]:
"""
Using the readStream as have to read data in stream from the json_files folder

"""

json_df = spark.readStream.option("maxFilesPerTrigger",5).schema(sc).format('json').option('header','true').load(json_file_path)


In [6]:
"""
If its value is true then stream reading has started and we can perform transformation

"""


json_df.isStreaming


True

In [7]:
"""

From nested to flat and dropping and remaining some of the columns


"""

df = json_df


df = df.withColumn('clicks',df.analytics.clicks)

df = df.withColumn('impressions',df.analytics.impressions)

df = df.withColumn('quantity',df.sales.quantity)
df = df.withColumn('total_price',df.sales.total_price)

df = df.drop("analytics","sales")

#f_df = df.select('datetime','clicks','impressions','quantity','total_price')

#f_df.schema


In [8]:
"""

Writing the data in stream as it is coming to up in CSV format.

3 modes to write in append(only new data), complete(old data and new coming), update(updating the rows)

"""


final_df = df

final_df.writeStream.outputMode("append").format("csv").option("path", csv_file_path).option("checkpointLocation", csv_checkpoint_path).option("header", "true").start()



22/09/06 11:40:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [ ]:
#spark.stop()